<a href="https://colab.research.google.com/github/HofstraDoboli/TextMining_F22/blob/main/read_inaugural_addresses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nltk
nltk.download('inaugural')
nltk.download('punkt')
from nltk import word_tokenize
# Uncomment the lines below if you get an error
# import nltk
# nltk.download('inaugural')
from nltk.corpus import inaugural

[nltk_data] Downloading package inaugural to /root/nltk_data...
[nltk_data]   Package inaugural is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# list names inaugural addresses
list_fnames = inaugural.fileids()

In [10]:
# methods inaugural object
print(dir(inaugural))

['CorpusView', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_citation', '_encoding', '_fileids', '_get_root', '_license', '_para_block_reader', '_read_para_block', '_read_sent_block', '_read_word_block', '_readme', '_root', '_sent_tokenizer', '_tagset', '_unload', '_word_tokenizer', 'abspath', 'abspaths', 'citation', 'encoding', 'ensure_loaded', 'fileids', 'license', 'open', 'paras', 'raw', 'readme', 'root', 'sents', 'words']


In [ ]:
print(inaugural.raw(list_fnames[0]))

In [5]:
list_docs = []    # list of raw text documents
for fid in list_fnames:
    list_docs.append(inaugural.raw(fid))

print(len(list_docs))
#print(name_doc)
#print(list_docs[0])

59


In [ ]:
# access the words
print(inaugural.words('1789-Washington.txt')) # replace words with paras - get paragraphs

['Fellow', '-', 'Citizens', 'of', 'the', 'Senate', ...]


In [6]:
from sklearn.feature_extraction.text import CountVectorizer # extract words and count words in each text
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter

import spacy   # another tokenizer, lemmatizer (has --> be)
nlp = spacy.load('en_core_web_sm')

In [31]:
def nlp_processing(doc):  # text processing for one text
    tokens = nlp(doc)

    #print(type(tokens))
    # eliminates stop words  and non alpha num
    terms = [token.lemma_.lower() for token in tokens if not token.is_stop and token.is_alpha]

    ners = []
    for ner in tokens.ents:
      tokens_ner = nlp(ner.text)
      if len(tokens_ner) > 1 and ner.label_ in ["PERSON", "GPE", "ORG","FAC"]:
        # remove stop words at the beginning
        if tokens_ner[0].is_stop:
          ner_text = ' '.join([t.text for t in tokens_ner[1:]])
        else:
          ner_text = ner.text
        ners.append(ner_text.lower())

    #print(len(ners)) # ners.keys())
    terms.extend(ners)

    return terms, ners

all_terms  = []
all_tokens = []
all_ners   = []
for doc in list_docs:
  terms_spacy, ners = nlp_processing(doc)
  all_ners.extend(ners)
  all_terms.extend(terms_spacy) # contains all words in the collection in one list
  #all_tokens.append(tokens_spacy) # contains all spacy tokens in each document, as a list of size num documents

#vectorizer = CountVectorizer(min_df = 1)
# stop_words = 'english', max_df, min_df, max_features, try analyzer = 'word'

#count_matrix = vectorizer.fit_transform(all_terms)

In [37]:
ner_freq_collec = Counter(all_ners)
sorted(ner_freq_collec)

['abraham lincoln',
 'adams and a',
 'amelia island',
 'american century',
 'american navy',
 'american states',
 'american wayâ\x80\x94restless',
 'america¡¦',
 'archibald macleish',
 'arlington cemetery',
 'arlington national cemetery',
 'articles of association',
 'articles of confederation',
 'articles of confederation and perpetual union',
 'atlantic states',
 'author of liberty',
 'barbary powers',
 'belleau wood',
 'beneficent creator',
 'board of engineers and naval commissioners',
 "britannic majesty's",
 'british government',
 'british house of commons',
 'british isles',
 'bureau of corporations',
 'buy american',
 'calvin coolidge',
 'capital of the nation',
 'cardinal bernardin',
 'chamber of congress',
 'chief executive',
 'chief justice',
 'chosin reservoir',
 'congress of the united states',
 'constitution the union of these states',
 'constitutional convention',
 'consulting board',
 'democratic party',
 'department of agriculture',
 'department of commerce and labor',

In [38]:
# compute the frequency of all terms in the collection
term_freq_collec = Counter(all_terms)
# how many terms there are in the collection
print("Size of the collection vocabulary", len(term_freq_collec))
# length of the collection = sum length of all documents
print("Length collection", len(all_terms))

Size of the collection vocabulary 6672
Length collection 58643


In [35]:
query = "cold war Soviet Union Russia China"
terms_query, tokens_query = nlp_processing(query)
str(terms_query)

"['cold', 'war', 'soviet', 'union', 'russia', 'china', 'soviet union']"

In [36]:
query_collect_freq = []
for w in terms_query:
  if w in term_freq_collec:
    query_collect_freq.append(term_freq_collec[w])
  else:
    query_collect_freq.append(0)

query_collect_freq


[6, 207, 3, 191, 1, 2, 3]

UP TO HERE

In [7]:
# connect to your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/TextMining_F22

/content/drive/MyDrive/TextMining_F22


In [9]:
# save your data
import pickle as pl

f_counter = open('file_counter.pkl', 'wb')
pl.dump(count_matrix, f_counter)
f_counter.close()
f_tokens  = open('file_tokens.pkl','wb')
pl.dump(all_tokens, f_tokens)
f_tokens.close()
f_terms   = open('file_terms.pkl', 'wb')
pl.dump(all_terms, f_terms)
f_terms.close()

NameError: ignored

In [12]:
import pickle as pl
# load your data
f_counter = open('file_counter.pkl', 'rb')
count_matrix1 = pl.load(f_counter)
f_counter.close()
f_tokens  = open('file_tokens.pkl','rb')
all_tokens1 = pl.load(f_tokens)
f_tokens.close()
f_terms   = open('file_terms.pkl', 'rb')
all_terms1 = pl.load(f_terms)
f_terms.close()


EOFError: ignored

In [26]:
count_matrix.shape

(58189, 6491)

In [16]:
count_matrix['cold']

IndexError: ignored

In [ ]:
#dir(vectorizer)
vocabulary = vectorizer.get_feature_names()
print(len(vocabulary))
print(vocabulary[:20])

3764
['-PRON-', 'abandon', 'abandonment', 'abide', 'ability', 'able', 'abode', 'abolish', 'aboriginal', 'abound', 'abraham', 'abreast', 'abroad', 'absence', 'absent', 'absolute', 'absolutely', 'absorb', 'abstain', 'abundance']


In [ ]:
# find the document frequency of each term from count_matrix
import scipy
import numpy as np
#from scipy.sparse.csr_matrix import sum
binary_matrix = count_matrix > 0
doc_freq = np.sum(binary_matrix, axis = 0)
print(doc_freq.shape)
doc_freq = np.asarray(doc_freq).ravel()  # converts a matrix to an array same as doc_freq.A1
print(doc_freq.shape)
print(doc_freq[:5])

(1, 3764)
(3764,)
[ 5 12  4 12 20]


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc_info = nlp(inaugural.raw('1993-Clinton.txt')) # '1789-Washington.txt'))

In [ ]:
for ent in doc_info.ents:
    e
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

today 20 25 DATE
American 54 62 NORP
winter 110 116 DATE
the spring 186 196 DATE
spring 200 206 DATE
America 300 307 GPE
America 343 350 GPE
America 429 436 GPE
America 522 529 GPE
American 668 676 NORP
American 714 722 NORP
Bush 784 788 PERSON
half-century 798 810 DATE
America 825 832 GPE
millions 849 857 CARDINAL
Today 960 965 DATE
the Cold War 1005 1017 EVENT
George Washington 1366 1383 PERSON
first 1384 1389 ORDINAL
America 1785 1792 GPE
today 1793 1798 DATE
Earth 1850 1855 LOC
millions 2067 2075 CARDINAL
Americans 2079 2088 NORP
millions 2395 2403 CARDINAL
Americans 2778 2787 NORP
today 2871 2876 DATE
the Civil War 2949 2962 EVENT
the Great Depression 2967 2987 EVENT
Thomas Jefferson 3128 3144 PERSON
Americans 3271 3280 NORP
America 3436 3443 GPE
America 3487 3494 GPE
today 3504 3509 DATE
American 3581 3589 NORP
America 3619 3626 GPE
America 4445 4452 GPE
America 4775 4782 GPE
Americans 5134 5143 NORP
today 5177 5182 DATE
America 5458 5465 GPE
Franklin Roosevelt 5522 5540 PERSON
A